In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import the libraries
import fitz
import os
from tqdm.auto import tqdm


In [3]:
pdf_path = './'
pdf_files = [f for f in os.listdir(pdf_path) if f.endswith(".pdf")]
print(pdf_files)

['Standford NLP.pdf', 'Attn_paper.pdf', 'Faiss.pdf', 'Gemini_paper.pdf', 'BART.pdf']


In [21]:
# Get Text content of the pdf
# For each pdf , get the pdf file contents 
# Store meta-data information about each pdf.

def format_text(text):
    text = text.strip("\n\n")
    text=text.replace("\n",'')
    return text

# # https://community.adobe.com/t5/acrobat-discussions/page-number-in-print-does-not-display-in-adobe-s-page-number-box/td-p/13781534
# Doesn't use Logical Page numbers. Use the normal Page numbers
# Logical causes issues during the rendering as you can't generalize to multiple pdfs

def get_text_content(pdf_text_, pdf_):
    doc = fitz.open(pdf_)
    print(len(doc))
    for page_num, page in enumerate(doc):
        # Extract the text content of the page
        text = page.get_text()
        text = format_text(text)
        name = pdf_.strip('.pdf')
        pdf_text_.append((text, page_num+1, name))
    return pdf_text_


In [22]:
books_ = []
books_ = [get_text_content([], pdf_) for pdf_ in pdf_files]

577
15
21
62
10


In [24]:
len(books_[1][7][0]) , books_[1][7]

(3107,
 ('Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on theEnglish-to-German and English-to-French newstest2014 tests at a fraction of the training cost.ModelBLEUTraining Cost (FLOPs)EN-DEEN-FREN-DEEN-FRByteNet [18]23.75Deep-Att + PosUnk [39]39.21.0 · 1020GNMT + RL [38]24.639.922.3 · 10191.4 · 1020ConvS2S [9]25.1640.469.6 · 10181.5 · 1020MoE [32]26.0340.562.0 · 10191.2 · 1020Deep-Att + PosUnk Ensemble [39]40.48.0 · 1020GNMT + RL Ensemble [38]26.3041.161.8 · 10201.1 · 1021ConvS2S Ensemble [9]26.3641.297.7 · 10191.2 · 1021Transformer (base model)27.338.13.3 · 1018Transformer (big)28.441.82.3 · 1019Residual DropoutWe apply dropout [33] to the output of each sub-layer, before it is added to thesub-layer input and normalized. In addition, we apply dropout to the sums of the embeddings and thepositional encodings in both the encoder and decoder stacks. For the base model, we use a rate ofPdrop = 0.1.Label SmoothingDuring training, we employed l

In [25]:
# Books_  -> [List[List[(text_content, page_num)]]

# Create a dictionary to store each pages documents into sentences

# Estimating that each token is 4 characters
# Page Info
def page_formatter(page):
    page_ = {}
    for pg in page:
        page_['doc_name'] = page[2]
        page_['text'] = page[0]
        page_['pg_num'] = page[1]
        page_['pg_num_chars'] = len(page[0])
        page_['pg_num_words'] = len(page[0].split(' '))
        page_['pg_num_sentences'] = len(page[0].split('. ')) # Since sentences usually begin with '. '
        page_['pg_num_tokens'] = page_['pg_num_chars'] / 4
    return page_
                                        
# Testing with a single page
# page_formatter(books_[1][6])

all_content_ = [[page_formatter(page) for page in books] for books in books_]


In [73]:
all_content_[3][3] # List[List[dictionaries]]


{'doc_name': 'Gemini_paper',
 'text': 'Gemini: A Family of Highly Capable Multimodal ModelsFigure 2 | Gemini supports interleaved sequences of text, image, audio, and video as inputs (illustratedby tokens of different colors in the input sequence). It can output responses with interleaved imageand text.tasks that require fine-grained understanding. In addition, Gemini can directly ingest audio signals at16kHz from Universal Speech Model (USM) (Zhang et al., 2023) features. This enables the model tocapture nuances that are typically lost when the audio is naively mapped to a text input (for example,see audio understanding demo on the website).Training the Gemini family of models required innovations in training algorithms, dataset, andinfrastructure. For the Pro model, the inherent scalability of our infrastructure and learning algorithmsenable us to complete pretraining in a matter of weeks, leveraging a fraction of the Ultra’s resources.The Nano series of models leverage additional ad

In [45]:
# import pandas as pd
# df_p1_mean = pd.DataFrame(all_content_[0])['pg_num_tokens'].mean()
# # df_p2 = pd.DataFrame(all_content_[1])
# # df_p3 = pd.DataFrame(all_content_[2])
# # df_p4 = pd.DataFrame(all_content_[3])
# print(df_p1_mean)

# Calculate the mean number of sentences in each document.Then use sentence_splitter 
# to split the long sentences
def calc_mean(doc):
    doc_df = pd.DataFrame(doc)
    doc_mean = doc_df['pg_num_tokens'].mean()
    doc_sentences_mean = doc_df['pg_num_sentences'].mean()
    return doc_mean, doc_sentences_mean

for doc in all_content_:
    print("Token_mean , Sentences_mean",calc_mean(doc))

Token_mean , Sentences_mean (725.0064991334489, 32.85441941074524)
Token_mean , Sentences_mean (642.45, 17.0)
Token_mean , Sentences_mean (1178.2619047619048, 26.857142857142858)
Token_mean , Sentences_mean (581.2137096774194, 11.983870967741936)
Token_mean , Sentences_mean (1003.2, 18.7)


In [74]:
# Each sentence is long , the mean token size are above and won't fit the embedding model for tokenization
# Split sentences into smaller chunks

# Method to put all sentences in an array called sentences for making the splitting easier
def sentence_formatter_per_page(doc):
    for page in doc:
        page['sentences'] = [sentence for sentence in page['text'].split('. ')]
        

In [75]:
def sentence_formatter_all_books(books):
    for item in books:
        sentence_formatter_per_page(item)
    

In [79]:
sentence_formatter_all_booksall_content_[0][16]

{'doc_name': 'Standford NLP',
 'text': '2.1•REGULAR EXPRESSIONS9any number of spaces! The star here applies only to the space ␣ that precedes it,not to the whole sequence. With the parentheses, we could write the expression/(Column␣[0-9]+␣*)*/ to match the word Column, followed by a number andoptional spaces, the whole pattern repeated zero or more times.This idea that one operator may take precedence over another, requiring us tosometimes use parentheses to specify what we mean, is formalized by the operatorprecedence hierarchy for regular expressions. The following table gives the orderoperatorprecedenceof RE operator precedence, from highest precedence to lowest precedence.Parenthesis()Counters* + ? {}Sequences and anchorsthe ˆmy end$Disjunction|Thus,becausecountershaveahigherprecedencethansequences,/the*/ matches theeeee but not thethe. Because sequences have a higher prece-dence than disjunction, /the|any/ matches the or any but not thany or theny.Patterns can be ambiguous in anot